# Async query examples

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from astropy.table import vstack
from scipy.optimize import leastsq, minimize

from scipy.stats import chisquare
import scienceplots
#from photometric_models import HG, HG12, HG1G2
from db import query, submit_job_list
#import emcee
#import corner
from tqdm import tqdm
import time

plt.style.use('science')

filters = ['g', 'r', 'i', 'z']
filter_colors = {'g': '#008060', 'r': '#ff4000', 'i': '#850000', 'z': '#6600cc'}
linestyles = {'g': "--", "r":"-.", "i":":", "z":"-"}
regions = {
    "NEO": "q <= 1.3",
    "Main Belt": "(mpc.e <1) and (mpc.q/(1-mpc.e) >= 1.6) and (mpc.q/(1-mpc.e) <= 4.2 and mpc.q>=1.3)",
    "Jupiter Trojans": "(mpc.q/(1-mpc.e) >= 5.05) and (mpc.q/(1-mpc.e) <= 5.35)" #5.05 and 5.35 AU
}

attribs = ['Chi2', 'G12', 'G12Err', 'H', 'HErr', 'Ndata']

In [13]:
%time
condition = regions["NEO"]

start = time.time()

results = submit_job_list(
    [f'''
        select distinct sss.ssObjectId
        from dp03_catalogs_10yr.SSSource as sss
        join dp03_catalogs_10yr.MPCORB as mpc on mpc.SSObjectId = sss.SSObjectId
        where {condition}
        ''']
)
complete = False
while complete == False:
    time.sleep(1)
    count = 0
    for result in list(results):
        if result.phase == "COMPLETED":
            #print(result.fetch_result())
            complete = True
            print(result.phase)

print(time.time() - start)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 2.86 µs
COMPLETED
14.29234004020691


In [14]:
start = time.time()
for result in list(results):
        if result.phase == "COMPLETED":
            print(result.fetch_result())
print(time.time() - start)

<Table length=41065>
     ssObjectId     
       int64        
--------------------
 5463436545848220655
 -718914334477707624
-8829936806506462420
 4341523755707737517
-2580115183126611617
 2202517760806173108
-1518175712613454268
-8618045748256124410
 6512273716856938023
-8912688653981837173
                 ...
 3188007418840501339
 2207505050739593347
 1130669389144427300
 6517153337351365225
-6375924284678216197
-4643596115910817481
-2853784656279912776
 1422036302698339752
-3261956336577558163
-8935622405846662221
1.7855689525604248


In [12]:
condition = regions["NEO"]

start = time.time()
for _, condition in regions.items():
    stmt = f'''
        select distinct sss.ssObjectId
        from dp03_catalogs_10yr.SSSource as sss
        join dp03_catalogs_10yr.MPCORB as mpc on mpc.SSObjectId = sss.SSObjectId
        where {condition}
        '''
    query(stmt)

print(time.time() - start)

729.8126583099365


In [12]:
for result in list(results):
    if result.phase == "COMPLETED":
        #print(result.fetch_result())
        print(result.phase)

COMPLETED
COMPLETED


In [13]:
mb_ids = list(pd.read_csv("./IDS/Main Belt_ids.csv", index_col=False)["ssObjectId"])
#table = None
delta = 10000
job_list = [
    f'''
        select sss.ssObjectId,
        band,
        count(phaseAngle), 
        min(phaseAngle) as minPhase, 
        max(phaseAngle) as maxPhase, 
        (max(phaseAngle) - min(phaseAngle))/count(sss.ssObjectId) as phaseDensity, 
        SUM(CASE WHEN phaseAngle < 2 THEN 1 ELSE 0 END) AS oppCOV, 
        SUM(CASE WHEN phaseAngle > 2 AND phaseAngle < 15 THEN 1 ELSE 0 END) AS modCOV, 
        SUM(CASE WHEN phaseAngle > 15 THEN 1 ELSE 0 END) AS highCOV,
        AVG(mag) as avgMag,
        STDDEV(mag) as magSTD,
        AVG(magErr) as avgMagErr,
        STDDEV(magErr) as MagErrSTD
        from dp03_catalogs_10yr.SSSource as sss
        left join dp03_catalogs_10yr.DiaSource as dia on sss.diaSourceId = dia.diaSourceId
        where sss.ssObjectId in {tuple(mb_ids[i*delta:(i+1)*delta])}
        group by sss.ssObjectId, band
        ''' for i in range(np.floor(len(mb_ids)/delta).astype(int) + 1)
]
results = submit_job_list(
    job_list
)

In [52]:
mb_ids = list(pd.read_csv("./IDS/Main Belt_ids.csv", index_col=False)["ssObjectId"])
#table = None
delta = 10000
photo_job_list = [
    f'''
    select sso.ssObjectId,
    g_Chi2, g_G12, g_G12Err, g_H, g_HErr, g_Ndata, 
    i_Chi2, i_G12, i_G12Err, i_H, i_HErr, i_Ndata, 
    r_Chi2, r_G12, r_G12Err, r_H, r_HErr, r_Ndata, 
    z_Chi2, z_G12, z_G12Err, z_H, z_HErr, z_Ndata
    from dp03_catalogs_10yr.SSObject as sso
    where sso.ssObjectId in {tuple(mb_ids[i*delta:(i+1)*delta])}
    ''' for i in range(np.floor(len(mb_ids)/delta).astype(int) + 1)
]
photo_results = submit_job_list(
    photo_job_list
)

In [55]:
queue = 0
completed = 0

for result in tqdm(list(photo_results)):
    result.executionduration = 600
    try:
        if result.phase == "COMPLETED":
            completed += 1
            #print(result.phase)
            #print(result.fetch_result())
        else:
            #print(result.phase)
            #http://docs.g-vo.org/pyvo/html/page013.html
            queue += 1
    except:
        print("Error with connection")
print(f"Completed {completed} jobs")

100%|██████████| 428/428 [07:21<00:00,  1.03s/it]

Completed 428 jobs


In [57]:
table = None

for result in tqdm(list(photo_results)):
    result.executionduration = 600
    try:
        if result.phase == "COMPLETED":
            if not table:
                table = result.fetch_result().to_table()
            else:
                table = vstack([table, result.fetch_result().to_table()])
            table.to_pandas().to_csv("./photometric/main_belt.csv")
        else:
            #print(result.phase)
            #http://docs.g-vo.org/pyvo/html/page013.html
            queue += 1
            print("Error")
    except:
        print("Error with connection")

100%|██████████| 428/428 [2:27:22<00:00, 20.66s/it]  


In [48]:
queue = 0
completed = 0

for result in tqdm(list(results)):
    result.executionduration = 600
    try:
        if result.phase == "COMPLETED":
            completed += 1
            #print(result.phase)
            #print(result.fetch_result())
        else:
            #print(result.phase)
            #http://docs.g-vo.org/pyvo/html/page013.html
            queue += 1
    except:
        print("Error with connection")
print(f"Completed {completed} jobs")

  0%|          | 0/428 [00:00<?, ?it/s]

100%|██████████| 428/428 [06:31<00:00,  1.09it/s]

Completed 428 jobs


In [51]:
table = None

for result in tqdm(list(results)):
    result.executionduration = 600
    try:
        if result.phase == "COMPLETED":
            if not table:
                table = result.fetch_result().to_table()
            else:
                table = vstack([table, result.fetch_result().to_table()])
            table.to_pandas().to_csv("./phase_parameters/main_belt.csv")
        else:
            #print(result.phase)
            #http://docs.g-vo.org/pyvo/html/page013.html
            queue += 1
            print("Error")
    except:
        print("Error with connection")

100%|██████████| 428/428 [5:34:52<00:00, 46.94s/it]  
